In [ ]:
from pygeodyn.PYGEODYN import Pygeodyn
from pygeodyn.pygeodyn_plot_scalingfactors import *

import datetime


# Prep

In [1]:
run_list = [
                'msis2',
#                 'jb2008',
#                 'dtm2020_o',
#                 'tiegcm_oc',
#                 'ctipe_oc',
#####                 'hasdm_oc',
           ]


yaxis_range = [-13.7 -.55 ,  -12.6+.25]# ] #full_fig.layout.yaxis2.range


CD_type = '30hrarc_scale3hr'

plot_dir='/data/SatDragModelValidation/notebooks/Paper2023_icesat2_assessment/plots/'

dir_modeldat='/data/SatDragModelValidation/data/inputs/atmos_models'
run_dict={}
for i in run_list:
    if i =='msis2':
        run_dict[i]={}
        run_dict[i]['num'] = 5 
        run_dict[i]['model_path'] = None

    if i =='dtm2020_o':
        run_dict[i]={}
        run_dict[i]['num'] = 3 
        run_dict[i]['model_path'] = None

    if i =='jb2008':
        run_dict[i]={}
        run_dict[i]['num'] = 1 
        run_dict[i]['model_path'] = None

print(run_dict)

{'msis2': {'num': 5, 'model_path': None}}


## Construct Arc Times

In [ ]:
timestart = pd.to_datetime('2018-11-09 00:00:00', format='%Y-%m-%d %H:%M:%S')
# timeend   = pd.to_datetime('2018-11-23 00:00:00', format='%Y-%m-%d %H:%M:%S')
timeend   = pd.to_datetime('2018-11-11 00:00:00', format='%Y-%m-%d %H:%M:%S')

arc_list         = []
epoch_start_24hcenter = []
epoch_end_24hcenter   = []

itime = timestart
while itime < timeend:
               
    itime_0 = itime
    itime = itime + pd.to_timedelta(24,'h')
    
    arc_list.append(itime_0.strftime('%Y.%j'))
    epoch_start_24hcenter.append(itime_0.strftime('%Y-%m-%d %H:%M:%S'))
    epoch_end_24hcenter.append(itime.strftime(  '%Y-%m-%d %H:%M:%S'))

    print(' ',itime_0.strftime('%Y.%j'),itime_0, ' to ', itime)        

    
##### update the epoch start to be 3-hours before and epoch end to be 3 hours later

epoch_start_list = []
epoch_end_list   = []


for i in epoch_start_24hcenter:
    itime = pd.to_datetime(i) - pd.to_timedelta(3,'h')
    epoch_start_list.append(itime.strftime('%Y-%m-%d %H:%M:%S'))

for i in epoch_end_24hcenter:
    itime = pd.to_datetime(i) + pd.to_timedelta(3,'h')
    epoch_end_list.append(itime.strftime('%Y-%m-%d %H:%M:%S'))
 
    
print(arc_list)
print()
print(epoch_start_list)
print()
print(epoch_end_list)

## Run Orbit Fits

In [ ]:
%load_ext autoreload
%autoreload 2

from gc import collect as gc_collect
import pickle 
import os


file_raw_ICs = f"/data/SatDragModelValidation/data/inputs/sat_icesat2/g2b/"\
          +"ICESat2_RawEphem_20181108_20181124.txt"


dir_save    =  '/data/SatDragModelValidation/data/outputs_clean/'\
             + 'icesat2/Paper2023_icesat2_assessment/results_2weeks_30harc_3hScaled/'


    
if CD_type == '30hrarc_scale3hr':
    cd_adjustment_boolean = True 
    hours_between_cd_adj  = 3
    file_string           = '30harc_3hScaled'

    pickleName = '_2weeks_30harc_3hScaled.pkl'
    
obj = {}
for i,den in enumerate(run_list):
    settings_icesat2= {
                 ### Basic input settings
                 'satellite'      : {'input': 'icesat2'},
                 'den_model'      : {'input': den},
                 'run_type'       : {'input': 'DataReduction_PCE'},
                 'run_specifier'  : {'input': '_30harc_3hScaled'},
                 'cd_model'       : {'input': 'BWDRAG'},
                 'file_string'    : {'input': file_string},
                 'model_data_path': {'input': run_dict[den]['model_path']},
                 ### Force Model settings
                  'cd_type'               : {'input':"Fixed_CD"},
                  'cd_value'              : {'input': 2.500000},
                  'scaling_factor'        : {'input': False},  # (dria)
                  #### Comment for unadjusted run:
                  'cd_adjustment_boolean' : {'input':cd_adjustment_boolean },
                  'hours_between_cd_adj'  : {'input':hours_between_cd_adj  },
                  ### ---------------------------------------
                  ### Run
                  'step'           : {'input': 10.},
                  'orbfil_step'    : {'input': 120.},
                  'which_ICfile'   : {'input':file_raw_ICs},
                  ###
                  'arc'            : {'input':arc_list},
                  'epoch_start'    : {'input':epoch_start_list},
                  'epoch_stop'     : {'input':epoch_end_list},  
                                
                  'global_options' : {'input':'pso_2018'},
                  ### Request read on raw outputs
                  'request_data'   : {'input': ['Trajectory_orbfil', 
                                               'Density', 
                                               'Residuals_summary',
                                               'DragFile',
                                               'AdjustedParams'
                                               ]},
              ### end dict
              }

    
    sat = Pygeodyn(settings_icesat2, use_file=False)
    sat.run_arcs()
#     obj[den] =  sat.getData_BigData_lowmemory()
    gc_collect()



#     pickle_file = dir_save+den+pickleName
#     if not os.path.exists(pickle_file):
#         print('Must create pickle file...')
#         print('   ',  pickle_file)
#         print('   ', 'Reading Geodyn Data')

#         ### Load the data into an object
#         sat = Pygeodyn(settings_icesat2, use_file=False)
#         obj = sat.getData_BigData_lowmemory()
#         gc_collect()

#         #### Pickle the object to save it
#         print('   ', 'Saving pickle')
#         filehandler = open(pickle_file, 'wb') 
#         pickle.dump(vars(obj), filehandler)
#         filehandler.close()
#         obj = 0
#         print('   ', 'Saved pickle')

# obj = {}
# for i,model in enumerate(run_list):     
#     ### Load the data if the pickles exist
#     print()
#     print()
#     gc_collect()

#     pickle_file = dir_save+model+pickleName

#     filehandler = open(pickle_file, 'rb') 
#     obj[model] = pickle.load(filehandler)
#     filehandler.close()
#     print('Loaded data from pickle... ', model)

